In [2]:
#######################################
# Testing Blocks for Treemakers
# by Tianyu tz2263@columbia.edu, Feb 2018
#######################################

import os, sys, io, time
import numpy as np
from multihist import Histdd, Hist1d
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # enable if you have a retina display
import pandas as pd
from scipy.optimize import curve_fit, minimize
pd.options.mode.chained_assignment = None        # default='warn'
import warnings
warnings.filterwarnings('ignore')

from scipy.interpolate import interp1d, InterpolatedUnivariateSpline

from numpy import sqrt, exp, pi, square
from tqdm import tqdm
import hax

def plt_config(title = None, xbounds = None, ybounds = None, xlabel = None, ylabel = None, colorbar = False, sci = False):
    if sci:
        plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    if title != None: plt.title(title)
    if xbounds != None: plt.xlim(xbounds)
    if ybounds != None: plt.ylim(ybounds)
    if xlabel != None: plt.xlabel(xlabel)
    if ylabel != None:  plt.ylabel(ylabel)
    if isinstance(colorbar,str):
        plt.colorbar(label = colorbar)
    elif colorbar:
        plt.colorbar(label = '$Number\ of\ Entries$')
    else:
        pass

In [24]:
class Treemaker_Testing():
    '''A common testing ground for treemakers
       Just use a single run to do it
    '''
    __version__ = '0.1.0'
    
    def __init__(self, treemakers):
        self.hax_init()
        
        if isinstance(treemakers, list):
            self.treemakers = treemakers
        else:
            self.treemakers = [treemakers]
        
        self.run_dict = dict(sciencerun0 = dict(Kr83m = '161114_1039', 
                                                none = '161122_2118', 
                                                AmBe = '161021_1514', 
                                                Rn220 = '161229_1111',),
                             sciencerun1 = dict(Kr83m = '170206_1255', 
                                                none = '170202_1747', 
                                                AmBe = '170316_0834', 
                                                Rn220 = '170313_1817', 
                                                neutron_generator = '170524_1435',
                                                Cs137 = '170404_1815',)
                            )
        
    def start_testing(self, sciencerun = 'sciencerun1', source_type = 'none', event_list_file = False, nevent_test = False):
        self.event_list_file = event_list_file
        
        if isinstance(nevent_test, int):
            tmp = pd.DataFrame(dict(event_number = list(range(nevent_test))))
            tmp.to_pickle(os.path.join(os.getcwd(), 'tmp.pkl'))
            self.event_list_file = os.path.join(os.getcwd(), 'tmp.pkl')
        
        if self.event_list_file:
            self.event_list = self.get_event_list(self.event_list_file)
        else:
            self.event_list = None
            
        self.test_run = self.run_dict[sciencerun][source_type]
        
        df, cut_history = hax.minitrees.load_single_dataset(run_id = self.test_run,
                                                            treemakers = self.treemakers,
                                                            preselection=None,
                                                            force_reload=False,
                                                            event_list = self.event_list,
                                                            )
        if isinstance(nevent_test, int): os.remove(os.path.join(os.getcwd(), 'tmp.pkl'))
        return df
    
    def hax_init(self, force_reload = False,
                 main_data_paths = ['/project2/lgrandi/xenon1t/processed/pax_v6.8.0'],
                 minitree_paths = ['/scratch/midway2/zhut/data/SingleScatter/data/minitrees']):

        if (not len(hax.config)) or force_reload:
            print ('Initiating hax, it takes some time...')
            # Initiate hax if not done already
            hax.init(experiment = 'XENON1T',
                     main_data_paths = main_data_paths,
                     minitree_paths = minitree_paths,
                     version_policy = 'loose',
                     make_minitrees = True,
                     minitree_caching = False, # We don't really use root anymore
                     pax_version_policy = '6.8.0',
                     tqdm_on = False,
                    )
            
    
    def get_event_list(self, event_list_file):
        return pd.read_pickle(event_list_file).event_number.values

In [25]:
from hax.treemakers.common import Basics
tt = Treemaker_Testing(Basics)

In [26]:
df = tt.start_testing(sciencerun = 'sciencerun1', 
                      source_type = 'Rn220', 
                      event_list_file = False, 
                      nevent_test = 10)


DEBUG:hax.__init__] Extraction completed, now concatenating data
DEBUG:hax.minitrees] Retrieved Basics minitree data for dataset 170313_1817


In [23]:
df

,drift_time,event_number,largest_coincidence,largest_other_s1,largest_other_s2,largest_unknown,largest_veto,run_number,s1,s1_area_fraction_top,s1_range_50p_area,s2,s2_area_fraction_top,s2_range_50p_area,x_pax,y_pax,z
0,NaN,0,0,180.162643,45.418507,7.300045,0,7859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,351550.0,1,0,27.827307,34006.609375,2.654929,0,7859,3447.929443,0.199824,80.738563,88019.000000,0.639695,1435.359731,23.882475,33.607662,-46.620014
2,669130.0,2,0,0.000000,5650.393555,2.987638,0,7859,25991.650391,0.094269,81.007177,986206.750000,0.648985,2599.801343,5.651263,-7.461193,-88.939819
3,NaN,3,0,1575.415649,40.157627,6.390357,0,7859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1430.0,4,0,0.000000,5968.052246,5.096013,0,7859,1080.870850,0.291950,80.035468,832700.187500,0.571561,313.417358,2.328274,48.891384,0.035979
5,733440.0,5,0,6.300565,121070.156250,7.546052,0,7859,14212.050781,0.115232,79.023229,424691.968750,0.630248,2590.670035,10.869463,44.764187,-97.509590
6,NaN,6,0,809.926880,56.068901,3.259259,0,7859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,732280.0,7,0,0.000000,53.483562,2.981774,0,7859,2422.950195,0.107052,72.664121,89.399147,0.520413,1892.290747,34.352982,9.843726,-97.355011
8,688240.0,8,0,6.851851,2397.627686,5.589493,0,7859,14010.063477,0.103409,86.404126,586061.687500,0.625178,3503.964765,-10.111876,31.756914,-91.486366
9,76460.0,9,0,7.779091,399.526337,11.324937,0,7859,3406.236084,0.332709,80.193394,96842.125000,0.629090,738.692111,27.668091,36.862988,-9.962305
